In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm import tqdm
import statsmodels as sm
from numpy import mean
from pdfquery import PDFQuery as pdfq
from lxml import etree

In [2]:
#Loading data
data = pd.read_csv('polls_us_election_2016.csv')
data.head()

,state,startdate,enddate,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,rawpoll_johnson,rawpoll_mcmullin,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin
0,U.S.,2016-11-03,2016-11-06,ABC News/Washington Post,A+,2220.0,lv,47.00,43.00,4.00,NaN,45.20163,41.72430,4.626221,NaN
1,U.S.,2016-11-01,2016-11-07,Google Consumer Surveys,B,26574.0,lv,38.03,35.69,5.46,NaN,43.34557,41.21439,5.175792,NaN
2,U.S.,2016-11-02,2016-11-06,Ipsos,A-,2195.0,lv,42.00,39.00,6.00,NaN,42.02638,38.81620,6.844734,NaN
3,U.S.,2016-11-04,2016-11-07,YouGov,B,3677.0,lv,45.00,41.00,5.00,NaN,45.65676,40.92004,6.069454,NaN
4,U.S.,2016-11-03,2016-11-06,Gravis Marketing,B-,16639.0,rv,47.00,43.00,3.00,NaN,46.84089,42.33184,3.726098,NaN


In [40]:
results = pdfq('federalelections2016.pdf')
results.load()

In [45]:
results

In [46]:
results.doc

In [47]:
results.doc.catalog

{'AcroForm': <PDFObjRef:114>,
 'MarkInfo': {'Marked': False},
 'Metadata': <PDFObjRef:2>,
 'OCProperties': {'D': {'OFF': [<PDFObjRef:1204>,
    <PDFObjRef:1081>,
    <PDFObjRef:958>,
    <PDFObjRef:830>,
    <PDFObjRef:709>,
    <PDFObjRef:584>,
    <PDFObjRef:1911>,
    <PDFObjRef:1794>,
    <PDFObjRef:1677>,
    <PDFObjRef:1556>],
   'Order': [[b'federalelections2016.pdf',
     <PDFObjRef:1210>,
     <PDFObjRef:1209>,
     <PDFObjRef:1208>,
     <PDFObjRef:1207>,
     <PDFObjRef:1206>,
     <PDFObjRef:1205>,
     <PDFObjRef:1204>,
     <PDFObjRef:1087>,
     <PDFObjRef:1086>,
     <PDFObjRef:1085>,
     <PDFObjRef:1084>,
     <PDFObjRef:1083>,
     <PDFObjRef:1082>,
     <PDFObjRef:1081>,
     <PDFObjRef:965>,
     <PDFObjRef:964>,
     <PDFObjRef:963>,
     <PDFObjRef:962>,
     <PDFObjRef:961>,
     <PDFObjRef:960>,
     <PDFObjRef:959>,
     <PDFObjRef:958>,
     <PDFObjRef:837>,
     <PDFObjRef:836>,
     <PDFObjRef:835>,
     <PDFObjRef:834>,
     <PDFObjRef:833>,
     <PDFObjRe

In [69]:
reslines = results.pq('LTTextLineHorizontal').contents()
reslines.html

<bound method PyQuery.html of ['Election Results for the ', 'U.S. President, the ', 'U.S. Senate and the ', 'U.S. House of ', 'Representatives ', 'FEDERAL ELECTIONS 2016 ', 'Election Results for the U.S. President, ', 'the U.S. Senate and the ', 'U.S. House of Representatives ', 'Federal Election Commission ', 'Washington, D.C. ', 'December 2017 ', ' ', 'Lisa J. Stevenson, Acting General Counsel ', '(Vacant), Inspector General ', 'Caroline C. Hunter, Vice Chair ', 'Lee E. Goodman ', 'Matthew S. Petersen ', 'Ellen L. Weintraub ', '(Vacant) ', '', <Element LTTextBoxHorizontal at 0x1863f415310>, '', <Element LTTextBoxHorizontal at 0x1863f6e38e0>, '', <Element LTTextBoxHorizontal at 0x1863fab4dc0>, '', <Element LTTextBoxHorizontal at 0x1863fab6440>, '', <Element LTTextBoxHorizontal at 0x1863fab7c00>, ' Public Disclosure and Media Relations Division ', ' Office of Communications ', ' 999 E Street, N.W. ', ' Washington, D.C. 20463 ', ' 800/424-9530 ', ' 202/694-1120 ', 'Eileen J. Leamon, Dep

In [93]:
type(results.pq('LTTextBoxHorizontal'))

pyquery.pyquery.PyQuery

In [98]:
resboxes = results.pq('LTTextBoxHorizontal:contains("Clinton")')
type(resboxes)

pyquery.pyquery.PyQuery

In [101]:
resboxes.text()

'\uf0b7 Map: 2016 Electoral Vote Distribution\n\uf0b7 Map: 2016 Presidential Popular Vote Differential\n\uf0b7 Map: 2016 Popular Vote: Trump\n\uf0b7 Map: 2016 Popular Vote: Clinton Hillary Clinton (Democrat, Women\'s Equality, Working Families) Clinton (D) Clinton (D)\n729,547\n116,454\n1,161,167\n380,494\n8,753,792\n1,338,870\n897,572\n235,603\n282,830\n4,504,975\n1,877,963\n266,891\n189,765\n3,090,729\n1,033,126\n653,669\n427,005\n628,854\n780,154\n357,735\n1,677,928\n1,995,196\n2,268,839\n1,367,716\n485,131\n1,071,068\n177,709\n284,494\n539,260\n348,526\n2,148,278\n385,234\n4,556,118\n2,189,316\n93,758\n2,394,164\n420,375\n1,002,106\n2,926,441\n252,525\n855,373\n117,458\n870,695\n3,877,868\n310,676\n178,573\n1,981,473\n1,742,718\n188,794\n1,382,536\n55,973\n65,853,514\n48.18% Clinton– 3\nTrump – 1 Clinton\n227 Clinton 2016 Popular Vote: Clinton Trump, Donald J.\nClinton, Hillary\nJohnson, Gary\nScattered\nStein, Jill Trump, Donald J.\nClinton, Hillary\nJohnson, Gary\nScattered\nStei

In [84]:
resboxes.find('Clinton, Hillary')

AttributeError: 'lxml.etree._ElementUnicodeResult' object has no attribute 'getchildren'

In [78]:
resboxes.html

<bound method PyQuery.html of ['', <Element LTTextLineHorizontal at 0x1863f45bd90>, <Element LTTextLineHorizontal at 0x1863f45a9e0>, <Element LTTextLineHorizontal at 0x1863f459b80>, <Element LTTextLineHorizontal at 0x1863f458c30>, <Element LTTextLineHorizontal at 0x1863f6de490>, '', <Element LTTextLineHorizontal at 0x1863e7f72a0>, <Element LTTextLineHorizontal at 0x1863ea92e90>, <Element LTTextLineHorizontal at 0x1863ea90230>, <Element LTTextLineHorizontal at 0x1863f368e10>, '', <Element LTTextLineHorizontal at 0x1863f361400>, <Element LTTextLineHorizontal at 0x1863f342260>, <Element LTTextLineHorizontal at 0x1863ea07a20>, '', <Element LTTextLineHorizontal at 0x1863e7f5a90>, <Element LTTextLineHorizontal at 0x1863f3ec1e0>, '', <Element LTTextLineHorizontal at 0x1863f44b9d0>, <Element LTTextLineHorizontal at 0x1863f423a70>, <Element LTTextLineHorizontal at 0x1863f414e60>, <Element LTTextLineHorizontal at 0x1863f417200>, <Element LTTextLineHorizontal at 0x1863f414870>, 'tatutory Officers

In [76]:
type(resboxes.html)

method

In [75]:
resboxes.html.find('Clinton, Hillary ')

AttributeError: 'function' object has no attribute 'find'

In [70]:
resboxes.text

<bound method PyQuery.text of ['', <Element LTTextLineHorizontal at 0x1863f45bd90>, <Element LTTextLineHorizontal at 0x1863f45a9e0>, <Element LTTextLineHorizontal at 0x1863f459b80>, <Element LTTextLineHorizontal at 0x1863f458c30>, <Element LTTextLineHorizontal at 0x1863f6de490>, '', <Element LTTextLineHorizontal at 0x1863e7f72a0>, <Element LTTextLineHorizontal at 0x1863ea92e90>, <Element LTTextLineHorizontal at 0x1863ea90230>, <Element LTTextLineHorizontal at 0x1863f368e10>, '', <Element LTTextLineHorizontal at 0x1863f361400>, <Element LTTextLineHorizontal at 0x1863f342260>, <Element LTTextLineHorizontal at 0x1863ea07a20>, '', <Element LTTextLineHorizontal at 0x1863e7f5a90>, <Element LTTextLineHorizontal at 0x1863f3ec1e0>, '', <Element LTTextLineHorizontal at 0x1863f44b9d0>, <Element LTTextLineHorizontal at 0x1863f423a70>, <Element LTTextLineHorizontal at 0x1863f414e60>, <Element LTTextLineHorizontal at 0x1863f417200>, <Element LTTextLineHorizontal at 0x1863f414870>, 'tatutory Officers

In [ ]:
#Function to convert time
def convert_time_to_int(data, column):
    new_column = []
    for elem in data[column]:
        new_column.append(datetime.fromisoformat(elem).timestamp())
    data[column] = new_column
    data[column] = data[column] - min(new_column)
    return min(new_column)
min_start_date = convert_time_to_int(data, 'startdate')
min_end_date = convert_time_to_int(data, 'enddate')

In [ ]:
datetime.fromisoformat('2016-11-03').timestamp()

In [ ]:
data

In [ ]:
startdates=data['startdate']
len(startdates.unique()) / len(startdates)

In [ ]:
(31446000.0 - 31359600.0) / 3

In [ ]:
states = data['state'].unique()
zeroes = [0]*len(states)
final_ravs = pd.DataFrame(data = zip(states, zeroes, zeroes, zeroes), columns = ['states', 'clinton', 'trump', 'johnson'])
roll_size = 10

In [ ]:
fig, axs = plt.subplots(57, figsize=(15, 500))
counter = 0
for state in data['state'].unique():
    d2 = data[data['state'] == state].sort_values(by="enddate").copy()
    d2 = d2[(d2['grade'] == 'A+') | (d2['grade'] == 'A') | (d2['grade'] == 'A-') | (d2['grade'] == 'B+') | (d2['grade'] == 'B') | (d2['grade'] == 'B-')]
    d2['rolling_clinton'] = d2[['adjpoll_clinton']].rolling(window=roll_size, center=True, min_periods=2).mean()['adjpoll_clinton']
    d2['rolling_trump'] = d2[['adjpoll_trump']].rolling(window=roll_size, center=True, min_periods=2).mean()['adjpoll_trump']
    d2['rolling_johnson'] = d2[['adjpoll_johnson']].rolling(window=roll_size, center=True, min_periods=2).mean()['adjpoll_johnson']

    final_ravs.loc[final_ravs['states'] == state, 'clinton'] = pd.to_numeric(mean(d2[['rawpoll_clinton']][-roll_size:]))
    final_ravs.loc[final_ravs['states'] == state, 'trump'] = pd.to_numeric(mean(d2[['rawpoll_trump']][-roll_size:]))
    final_ravs.loc[final_ravs['states'] == state, 'johnson'] = pd.to_numeric(mean(d2[['rawpoll_johnson']][-roll_size:]))
    
    axs[counter].plot(d2['enddate'], d2['rolling_clinton'], color="blue")
    axs[counter].plot(d2['enddate'], d2['rolling_trump'], color='red')
    axs[counter].plot(d2['enddate'], d2['rolling_johnson'], color='yellow')
    axs[counter].scatter(d2['enddate'], d2['adjpoll_clinton'], color="blue")
    axs[counter].scatter(d2['enddate'], d2['adjpoll_trump'], color="red")
    axs[counter].scatter(d2['enddate'], d2['adjpoll_johnson'], color="yellow")
    axs[counter].axhline(y=50, color="black", linestyle="-")
    axs[counter].axvline(x=(datetime.fromisoformat('2016-08-17').timestamp() - min_start_date), color="black", linestyle="-")
    axs[counter].set_title(state)
    axs[counter].set_ylim([0,100])
    counter += 1

In [ ]:
final_ravs

American elections work on the electoral college, therefore the prediction for who would win needs to include some kind of simulation of the electoral college.
https://en.wikipedia.org/wiki/United_States_Electoral_College
This can be simulated as P(R >= 270) = \sum{x \in ec_state} P(R_x > 50) or P(D > 270) = \sum{x \in ec_state} P(D_x >= 50)

In [ ]:
ecr = {
    "Alabama" : 9,
    "Alaska" : 3,
    "Arizona" : 11,
    "Arkansas" : 6,
    "California" : 54,
    "Colorado" : 10,
    "Connecticut" : 7,
    "Delaware" : 3,
    "District of Columbia" : 3,
    "Florida" : 30,
    "Georgia" : 16,
    "Hawaii" : 4,
    "Idaho" : 4,
    "Illinois" : 19,
    "Indiana" : 11,
    "Iowa" : 6,
    "Kansas" : 6,
    "Kentucky" : 8,
    "Louisiana" : 8,
    "Maine" : 2,
    "Maine CD-1" : 1,
    "Maine CD-2" : 1,
    "Maryland" : 10,
    "Massachusetts" : 11,
    "Michigan" : 15,
    "Minnesota" : 10,
    "Mississippi" : 6,
    "Missouri" : 10,
    "Montana" : 4,
    "Nebraska" : 2,
    "Nebraska CD-1" : 1,
    "Nebraska CD-2" : 1,
    "Nebraska CD-3" : 1,
    "Nevada" : 6,
    "New Hampshire" : 4,
    "New Jersey" : 14,
    "New Mexico" : 5,
    "New York" : 28,
    "North Carolina" : 16,
    "North Dakota" : 3,
    "Ohio" : 17,
    "Oklahoma" : 7,
    "Oregon" : 8,
    "Pennsylvania" : 19,
    "Rhode Island" : 4,
    "South Carolina" : 9,
    "South Dakota" : 3,
    "Tennessee" : 11,
    "Texas" : 40,
    "Utah" : 6,
    "Vermont" : 3,
    "Virginia" : 13,
    "Washington" : 12,
    "West Virginia" : 4,
    "Wisconsin" : 10,
    "Wyoming" : 3
}
#https://en.wikipedia.org/wiki/2012_United_States_presidential_election
past_elections = {
    "U.S." : 0.5106,
    "Alabama" : 0.3836,
    "Alaska" : 0.4081,
    "Arizona" : 0.4459,
    "Arkansas" : 0.3688,
    "California" : 0.6024,
    "Colorado" : 0.5149,
    "Connecticut" : 0.5806,
    "Delaware" : 0.5861,
    "District of Columbia" : 0.9091,
    "Florida" : 0.5001,
    "Georgia" : 0.4548,
    "Hawaii" : 0.7055,
    "Idaho" : 0.3204,
    "Illinois" : 0.5760,
    "Indiana" : 0.4393,
    "Iowa" : 0.5199,
    "Kansas" : 0.3805,
    "Kentucky" : 0.3780,
    "Louisiana" : 0.4058,
    "Maine" : 0.5627,
    "Maine CD-1" : 0.5957,
    "Maine CD-2" : 0.5294,
    "Maryland" : 0.6197,
    "Massachusetts" : 0.6065,
    "Michigan" : 0.5421,
    "Minnesota" : 0.5265,
    "Mississippi" : 0.4379,
    "Missouri" : 0.4438,
    "Montana" : 0.4170,
    "Nebraska" : 0.3803,
    "Nebraska CD-1" : 0.4083,
    "Nebraska CD-2" : 0.4570,
    "Nebraska CD-3" : 0.2782,
    "Nevada" : 0.5236,
    "New Hampshire" : 0.5198,
    "New Jersey" : 0.5838,
    "New Mexico" : 0.5299,
    "New York" : 0.6335,
    "North Carolina" : 0.4835,
    "North Dakota" : 0.3870,
    "Ohio" : 0.5067,
    "Oklahoma" : 0.3323,
    "Oregon" : 0.5424,
    "Pennsylvania" : 0.5197,
    "Rhode Island" : 0.6270,
    "South Carolina" : 0.4409,
    "South Dakota" : 0.3987,
    "Tennessee" : 0.3908,
    "Texas" : 0.4138,
    "Utah" : 0.2469,
    "Vermont" : 0.6657,
    "Virginia" : 0.5116,
    "Washington" : 0.5616,
    "West Virginia" : 0.3554,
    "Wisconsin" : 0.5283,
    "Wyoming" : 0.2782
}

In [ ]:
adj_data = data.copy()
result = []
for state in data['state']:
    result.append(past_elections[state] > 0.5)
adj_data['Y'] = result
adj_data.head()

In [ ]:
print(data['rawpoll_clinton'].rolling(30))

In [ ]:
data['rawpoll_clinton'].rolling(30).mean()

In [ ]:
for elem in data[data['state'] != "U.S."].sort_values(by='startdate')['startdate']:
    print(elem)

In [ ]:
def find_latest_national_poll(df, current_time):
    min_diff = -1
    best_val = None
    for _, row in df.iterrows():
        time = row['startdate']
        new_diff = abs(current_time - time)
        if min_diff < 0:
            min_diff = new_diff
            best_val = row['rawpoll_clinton']
        elif min_diff > new_diff:
            min_diff = new_diff
            best_val = row['rawpoll_clinton']
    if best_val == None:
        return 0
    return best_val

def find_weighted_mean(poll, samplesize):
    somme = 0
    total = 0
    for i in range(len(poll)):
        somme += poll[i] * samplesize[i]
        total += samplesize[i]
    return somme / total

def find_weighted_variance(grades, samplesize, grades_dict):
    somme = 0
    total = 0
    for i in range(len(grades)):
        try:
            grade = grades_dict[grades[i]]
        except:
            grade = grades_dict['nan']
        somme += (samplesize[i] ** 2) * grade
        total += samplesize[i]
    return somme / total

def poll_transform1(df, ecr, past_elections, grades_dict, poll_range, column_c, column_t):
    all_poll_times = df[df['state'] != "U.S."].sort_values(by='enddate')['enddate'].unique()
    pbar = tqdm(total = len(all_poll_times), desc="Loading", position=0)
    times = []
    votes_timeline = []
    new_elections = past_elections.copy()
    for time in all_poll_times:
        times.append(time)
        votes = 0
        new_df = df[df['enddate'] <= time]
        new_df = new_df[new_df['enddate'] > (time - poll_range)]
        for state in ecr.keys():
            current_df = new_df[new_df['state'] == state]
            clinton = current_df[column_c]
            trump = current_df[column_t]
            if len(clinton) > 1 and len(trump) > 1:
                mean = clinton.mean() - trump.mean()
                var = clinton.var() + trump.var()
                if var != 0:
                    prob = 1 - stats.norm.cdf(0, loc=mean, scale=var)
                    votes += (prob > 0.5) * ecr[state]
                    new_elections[state] = prob
                else:
                    votes += ecr[state] * (new_elections[state] > 0.5)
            else:
                votes += ecr[state] * (new_elections[state] > 0.5)
            '''
                var = 0
                if len(clinton) > 1 and len(trump) > 1:
                    mean = clinton.mean() - trump.mean()
                    for elem in current_df['grade']:
                        try:
                            var += grades_dict[elem]
                        except:
                            var += 1
                    if var != 0:
                        prob = 1 - stats.norm.cdf(0, loc=mean, scale=var)
                        states_results[state] = prob
                        votes += prob * ecr[state]
                    else:
                        votes += ecr[state] * (mean / 100 > 0.5)
                else:
                    votes += ecr[state] * (past_elections[state] > 0.5)
            '''
            '''
            current_df = new_df[new_df['state'] == state].reset_index(drop=True)
            clinton = current_df['adjpoll_clinton']
            trump = current_df['adjpoll_trump']
            if len(clinton) > 1 and len(trump) > 1:
                mean = find_weighted_mean(current_df['adjpoll_clinton'], current_df['samplesize']) - find_weighted_mean(current_df['adjpoll_trump'], current_df['samplesize'])
                var = 2 * find_weighted_variance(current_df['grade'], current_df['samplesize'], grades_dict)
                if var != 0:
                    prob = 1 - stats.norm.cdf(0, loc=mean, scale=var)
                    votes += (prob > 0.5) * ecr[state]
                    new_elections[state] = prob
                else:
                    votes += ecr[state] * (new_elections[state] > 0.5)
            else:
                votes += ecr[state] * (new_elections[state] > 0.5)
            '''
        pbar.update(1)
        votes_timeline.append(votes)
    return pd.DataFrame({"times" : times, "votes_timeline" : votes_timeline}), new_elections

In [ ]:
grades_dict = {"A+" : 10 ** (-6),
               "A" : 10 ** (-5),
               "A-" : 10 ** (-4),
               "B+" : 10 ** (-3),
               "B" : 10 ** (-2),
               "B-" : 10 ** (-1),
               "C+" : 10 ** (-5),
               "C" : 10 ** (-4),
               "C-" : 10 ** (-3),
               "D" : 10 ** (-2),
               "nan" : 10 ** (-1)}
new_data = data[(data['grade'] == 'A+') | (data['grade'] == 'A') | (data['grade'] == 'A-') | (data['grade'] == 'B+') | (data['grade'] == 'B') | (data['grade'] == 'B-')]
votes_timeline_df, states_results = poll_transform1(new_data, ecr, past_elections, grades_dict, 28800.0 * 14, 'rawpoll_clinton', 'rawpoll_trump')

In [ ]:
plt.plot(votes_timeline_df['times'] / 28800.0, votes_timeline_df['votes_timeline'].rolling(10).mean(), color="blue")
plt.plot(votes_timeline_df['times'] / 28800.0, 538 - votes_timeline_df['votes_timeline'].rolling(10).mean(), color="red")
plt.axhline(y=270, color="black", linestyle="-")
#Don't know what happened, needs further investigation
#Seems to be when polling starts to ramp up
plt.axvline(x=(datetime.fromisoformat('2016-08-17').timestamp() - min_end_date) / 28800, color="black", linestyle="-")
#First presidential debate
plt.axvline(x=(datetime.fromisoformat('2016-09-26').timestamp() - min_end_date) / 28800, color="black", linestyle="-")
#Access Hollywood tapes
plt.axvline(x=(datetime.fromisoformat('2016-10-07').timestamp() - min_end_date) / 28800, color="black", linestyle="-")
#Comey investigation
plt.axvline(x=(datetime.fromisoformat('2016-10-28').timestamp() - min_end_date) / 28800, color="black", linestyle="-")

Election day was November 8th and the last poll was conducted on November 3rd (1478145600.0)

In [ ]:
states_results

In [ ]:
states = list(states_results.keys())
date = [1478145600.0] * len(states)

In [ ]:
pd.DataFrame({"state" : states, "startdate" : date}).to_csv('Predictions.csv', index=False)

In [ ]:
states

Trying to include samplesize and grade to weigh different polls and turn them into random variables

In [ ]:
def conv_poll_to_rv(data, index, grades_dict):
    mean = data['rawpoll_clinton'][index]
    variance = grades_dict[data['grades'][index]] / data['samplesize'][index]
    return mean, variance

In [ ]:
grades_dict = {}
for grade in data['grade'].unique():
    grades_dict[grade] = len(data[data['grade'] == str(grade)])
values = list(grades_dict.keys())
values[5] = "NONE"
plt.bar(values, list(grades_dict.values()))
plt.show()

In [ ]:
import requests
import re

In [ ]:
response = requests.get("https://projects.fivethirtyeight.com/pollster-ratings/")
response.status_code

In [ ]:
string = ''
for pollster in data['pollster'].unique():
    string += pollster + "|"
string = string[:-1]
string

In [ ]:
poll_values = re.findall("<[^<>]*>([^<>]*)<[^<>]*>", response.text)
new_poll_values = []
for elem in poll_values:
    if elem != "":
        new_poll_values.append(elem)

In [ ]:
pollsters_skew = {}
has_key = False
for elem in new_poll_values[10:]:
    if re.match(".*(" + string + ").*", elem) and not has_key:
        new_key = elem
        has_key = True
    elif re.match("^[R|D]\+\d\.\d$", elem) and has_key:
        has_key = False
        if elem[0] == 'R':
            pollsters_skew[new_key] = float(elem[2:])
        elif elem[0] == 'D':
            pollsters_skew[new_key] = -1 * float(elem[2:])
pollsters_skew

https://projects.fivethirtyeight.com/pollster-ratings/

In [ ]:
adjusted_raw_polls_clinton = []
adjusted_raw_polls_trump = []
for _, row in data.iterrows():
    try:
        adjusted_raw_polls_clinton.append(row['rawpoll_clinton'] + pollsters_skew[row['pollster']])
        adjusted_raw_polls_trump.append(row['rawpoll_trump'] - pollsters_skew[row['pollster']])
    except KeyError:
        adjusted_raw_polls_clinton.append(row['rawpoll_clinton'])
        adjusted_raw_polls_trump.append(row['rawpoll_trump'])
data['adjusted_raw_polls_clinton'] = adjusted_raw_polls_clinton 
data['adjusted_raw_polls_trump'] = adjusted_raw_polls_trump

In [ ]:
d3 = data[(data['grade'] == 'A+') | (data['grade'] == 'A') | (data['grade'] == 'A-') | (data['grade'] == 'B+') | (data['grade'] == 'B') | (data['grade'] == 'B-')]
votes_timeline_df, states_results = poll_transform1(d3, ecr, past_elections, grades_dict, 28800.0 * 7, 'adjusted_raw_polls_clinton', 'adjusted_raw_polls_trump')

In [ ]:
plt.plot(votes_timeline_df['times'] / 28800.0, votes_timeline_df['votes_timeline'].rolling(10).mean(), color="blue")
plt.plot(votes_timeline_df['times'] / 28800.0, 538 - votes_timeline_df['votes_timeline'].rolling(10).mean(), color="red")
plt.axhline(y=270, color="black", linestyle="-")
#Don't know what happened, needs further investigation
#Seems to be when polling starts to ramp up
plt.axvline(x=(datetime.fromisoformat('2016-08-26').timestamp() - min_start_date) / 28800, color="black", linestyle="-")
#First presidential debate
plt.axvline(x=(datetime.fromisoformat('2016-09-26').timestamp() - min_start_date) / 28800, color="black", linestyle="-")
#Access Hollywood tapes
plt.axvline(x=(datetime.fromisoformat('2016-10-07').timestamp() - min_start_date) / 28800, color="black", linestyle="-")
#Comey investigation
plt.axvline(x=(datetime.fromisoformat('2016-10-28').timestamp() - min_start_date) / 28800, color="black", linestyle="-")

In [ ]:
code = {'Alabama': 'AL',
        'Alaska': 'AK',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'District of Columbia': 'DC',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY'}
inv_code = {v: k for k, v in code.items()}
new_state_results = {}
for state in states_results.keys():
    if state != "U.S." and "CD" not in state:
        new_state_results[code[state]] = states_results[state]
states_results = new_state_results.copy()
states_results

In [ ]:
winner = {}
for state in states_results.keys():
    winner[state] = (states_results[state] > 0.5) + 0 
winner

In [ ]:
spec_ecr = {}
for state in ecr:
    if state != "U.S." and "CD" not in state:
        spec_ecr[state] = ecr[state]

In [ ]:
results = pd.DataFrame({"States" : list(states_results.keys()), "Result" : list(states_results.values())})
winner_results = pd.DataFrame({"States" : list(winner.keys()), "Result" : list(winner.values())})
spec_ecr = []
for state in results['States']:
    spec_ecr.append(ecr[inv_code[state]])

In [ ]:
codes = []
for state in results['States']:
    codes.append(inv_code[state])
results['State_code'] = codes
results

In [ ]:
import plotly.express as px

In [ ]:
fig = px.choropleth(results, 
                    locationmode='USA-states', 
                    locations="States", 
                    color="Result", 
                    scope="usa",
                    color_continuous_scale = ["red", "white", "blue"],
                    range_color=(0, 1))
fig.show()

In [ ]:
real_results = {'AL': 0.3436,
                'AK': 0.3635,
                'AZ': 0.4458,
                'AR': 0.3365,
                'CA': 0.6173,
                'CO': 0.4816,
                'CT': 0.5457,
                'DE': 0.5309,
                'DC': 0.9086,
                'FL': 0.4782,
                'GA': 0.4564,
                'HI': 0.6222,
                'ID': 0.2749,
                'IL': 0.5583,
                'IN': 0.3791,
                'IA': 0.3791,
                'KS': 0.3605,
                'KY': 0.3268,
                'LA': 0.3845,
                'ME': 0.4783,
                'MD': 0.6033,
                'MA': 0.6001,
                'MI': 0.4727,
                'MN': 0.4644,
                'MS': 0.4006,
                'MO': 0.3814,
                'MT': 0.3575,
                'NE': 0.3370,
                'NV': 0.4792,
                'NH': 0.4698,
                'NJ': 0.5545,
                'NM': 0.4826,
                'NY': 0.5901,
                'NC': 0.4617,
                'ND': 0.2723,
                'OH': 0.4356,
                'OK': 0.2893,
                'OR': 0.5007,
                'PA': 0.4746,
                'RI': 0.5441,
                'SC': 0.4067,
                'SD': 0.3174,
                'TN': 0.3472,
                'TX': 0.4324,
                'UT': 0.2746,
                'VT': 0.5668,
                'VA': 0.4973,
                'WA': 0.5254,
                'WV': 0.2643,
                'WI': 0.4645,
                'WY': 0.2188}

#https://en.wikipedia.org/wiki/2016_United_States_presidential_election

In [ ]:
error = {}

for key in real_results:
    error[key] = states_results[key] - real_results[key]

In [ ]:
error_df = pd.DataFrame({"States" : list(error.keys()), "Error" : list(error.values())})

In [ ]:
error_df['State_code'] = codes
error_df

In [ ]:
codes = []
for state in winner_results['States']:
    codes.append(inv_code[state])
winner_results['State_code'] = codes
results = []
for result in winner_results['Result']:
    if result == 0:
        results.append("Trump")
    else:
        results.append("Hillary")
winner_results["Result"] = results
winner_results

In [ ]:
fig = px.choropleth(error_df, 
                    locationmode='USA-states', 
                    locations="States", 
                    color="Error", 
                    scope="usa",
                    color_continuous_scale = ["blue", "white", "red"],
                    color_continuous_midpoint = 0)
fig.show()

In [ ]:
fig = px.choropleth(winner_results, 
                    locationmode='USA-states', 
                    locations="States", 
                    color="Result", 
                    scope="usa",
                    labels="Change of Hillary Clinton of winning the state in the electoral college",
                    color_discrete_map = {"Trump" : "red", "Hillary" : "blue"})
fig.show()

In [ ]:
def map_date(df, date, date_type, column_c, column_t, min_grade = "ALL", continuous = True):
    all_poll_times = df.copy()
    date_formatted = datetime.fromisoformat(date).timestamp()
    
    grades = ["D-", "D", "D+", "C-", "C", "C+", "B-", "B", "B+", "A-", "A", "A+"]
    if min_grade != "ALL" and min_grade not in grades:
        raise Exception("Not valid grade")
    elif grades != "ALL":
        for grade in grades:
            if grade == min_grade:
                break
            all_poll_times = all_poll_times[(all_poll_times['grade'] != grade)]
            
    if not (date_type == 'startdate' or date_type == 'enddate'):
        raise Exception("NOT VALID DATE TYPE")
    if date_type == 'startdate':
        date_formatted -= min_start_date
        all_poll_times = all_poll_times[(all_poll_times['state'] != "U.S.") & (all_poll_times['startdate'] < date_formatted)].sort_values(by='startdate')['startdate'].unique()
    else:
        date_formatted -= min_end_date
        all_poll_times = all_poll_times[(all_poll_times['state'] != "U.S.") & (all_poll_times['enddate'] < date_formatted)].sort_values(by='enddate')['enddate'].unique()
    
    pbar = tqdm(total = len(all_poll_times), desc="Loading", position=0)
    new_elections = past_elections.copy()
    poll_range = 28800.0 * 7
    for time in all_poll_times:
        votes = 0
        new_df = df[df['enddate'] <= time]
        new_df = new_df[new_df['enddate'] > (time - poll_range)]
        for state in ecr.keys():
            current_df = new_df[new_df['state'] == state]
            clinton = current_df[column_c]
            trump = current_df[column_t]
            if len(clinton) > 1 and len(trump) > 1:
                mean = clinton.mean() - trump.mean()
                var = clinton.var() + trump.var()
                if var != 0:
                    prob = 1 - stats.norm.cdf(0, loc=mean, scale=var)
                    new_elections[state] = prob
        pbar.update(1)
    keys_to_remove = ['U.S.']
    for key in new_elections.keys():
        if 'CD' in key:
            keys_to_remove.append(key)
    for key in keys_to_remove:
        new_elections.pop(key)
    results_df = pd.DataFrame({"States" : list(new_elections.keys()), "Result" : list(new_elections.values())})
    
    states_code = []
    for state in results_df['States']:
        states_code.append(code[state])
    results_df['States'] = states_code
    if continuous:
        fig = px.choropleth(results_df, 
                    locationmode='USA-states', 
                    locations="States", 
                    color="Result", 
                    scope="usa",
                    color_continuous_scale = ["red", "white", "blue"],
                    range_color=(0, 1))
    else:
        winner = []
        for res in results_df['Result']:
            if res > 0.5:
                winner.append("Clinton")
            else:
                winner.append("Trump")
        results_df['Winner'] = winner
        fig = px.choropleth(results_df, 
                    locationmode='USA-states', 
                    locations="States", 
                    color="Winner", 
                    scope="usa",
                    color_discrete_map = {"Trump" : "red", "Clinton" : "blue"})
    fig.show()
    return results_df

In [ ]:
results_df = map_date(data, '2016-09-20', 'startdate', 'rawpoll_clinton', 'rawpoll_trump', min_grade = "B-", continuous = True)

In [ ]:
results_df = map_date(data, '2016-09-20', 'startdate', 'rawpoll_clinton', 'rawpoll_trump', min_grade = "B-", continuous = False)

In [ ]:
data['grade'] != "B"

In [ ]:
res = requests.get('https://web.archive.org/web/20161108055435/http://projects.fivethirtyeight.com/pollster-ratings/')

In [ ]:
res.text

In [ ]:
import re
res_values = []
for val in re.findall("<[^<>]*>([^<>]+)<[^<>]*>", res.text):
    if val.strip() != '\n' and len(val.strip()) > 0:
        res_values.append(val.strip())
res_values

In [ ]:
res_values[41:]

In [ ]:
counter = 0
updated_poll_values = []
for val in res_values[41:]:
    if val == '✖':
        continue
    elif counter % 8 == 0:
        if counter != 0:
            updated_poll_values.append(current_poll_values)
        current_poll_values = {
            "pollster" : val
        }
    elif counter % 8 == 1:
        try:
            test = int(val)
            current_poll_values['polls analyzed'] = val
        except:
            current_poll_values['pollster'] += val
            counter -= 1
    elif counter % 8 == 2:
        current_poll_values['simple average error'] = val
    elif counter % 8 == 3:
        current_poll_values['races called correctly'] = val
    elif counter % 8 == 4:
        current_poll_values['advanced'] = val
    elif counter % 8 == 5:
        current_poll_values['predictive'] = val
    elif counter % 8 == 6:
        current_poll_values['538 grade'] = val
    elif counter % 8 == 7:
        if re.search('[D|R].\d+.\d+', val) != None:
            current_poll_values['mean-reverted bias'] = val
        else:
            current_poll_values['mean-reverted bias'] = None
            counter -= 1
    counter += 1
updated_poll_values

In [ ]:
polls_df_updated = {}
for di in updated_poll_values:
    for key in di.keys():
        if key not in polls_df_updated.keys():
            polls_df_updated[key] = [di[key]]
        else:
            polls_df_updated[key] += [di[key]]
polls_df_updated = pd.DataFrame(polls_df_updated)
polls_df_updated.to_csv('Updated_poll_values.csv')
polls_df_updated

In [ ]:
counter = 0
updated_poll_values = {
    'pollster' : [],
    'polls analyzed' : [],
    'simple average error' : [],
    'races called correctly' : [],
    'advanced' : [],
    'predictive' : [],
    '538 grade' : [],
    'mean-reverted bias' : []
}
for val in res_values[41:]:
    if val == '✖':
        continue
    elif counter % 8 == 0:
        updated_poll_values['pollster'] += [val]
    elif counter % 8 == 1:
        try:
            test = int(val)
            updated_poll_values['polls analyzed'] += [val]
        except:
            
            updated_poll_values['pollster'][-1] += val
            counter -= 1
    elif counter % 8 == 2:
        updated_poll_values['simple average error'] += [val]
    elif counter % 8 == 3:
        updated_poll_values['races called correctly'] += [val]
    elif counter % 8 == 4:
        updated_poll_values['advanced'] += [val]
    elif counter % 8 == 5:
        updated_poll_values['predictive'] += [val]
    elif counter % 8 == 6:
        updated_poll_values['538 grade'] += [val]
    elif counter % 8 == 7:
        updated_poll_values['mean-reverted bias'] += [val]
    counter += 1
updated_poll_values